In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
import random

In [8]:
#read in csv and index teams based on id
df = pd.read_csv('euro2024_data.csv')
df = df.set_index('id',drop=True)
df

,id,squad,played90s_wc2022,avgpossession_wc2022,goalsx90_wc2022,assistsx90_wc2022,expgoalsx90_wc2022,diffgoalx90_wc2022,played90s_nl2022,avgpossession_nl2022,...,played90s_wc2018,avgpossession_wc2018,goalsx90_wc2018,assistsx90_wc2018,expgoalsx90_wc2018,diffgoalx90_wc2018,played90s_euro2016,avgpossession_euro2016,goalsx90_euro2016,assistsx90_euro2016
0,1,Albania,NaN,NaN,NaN,NaN,NaN,NaN,4.0,54.5,...,NaN,NaN,NaN,NaN,NaN,NaN,3.0,40.3,0.33,0.33
1,2,Austria,NaN,NaN,NaN,NaN,NaN,NaN,6.0,43.5,...,NaN,NaN,NaN,NaN,NaN,NaN,3.0,54.0,0.33,0.33
2,3,Belgium,3.0,57.0,0.33,0.33,1.57,-1.24,6.0,62.3,...,7.0,52.9,2.14,1.71,1.90,0.24,5.0,52.4,1.80,1.60
3,4,Croatia,7.7,54.3,1.04,1.04,0.91,0.13,8.7,51.3,...,8.0,56.7,1.62,1.00,1.40,0.22,4.3,48.8,1.15,0.92
4,5,Czech Republic,NaN,NaN,NaN,NaN,NaN,NaN,6.0,35.0,...,NaN,NaN,NaN,NaN,NaN,NaN,3.0,42.3,0.67,0.33
5,6,Denmark,3.0,60.0,0.33,0.33,0.90,-0.57,6.0,47.7,...,4.3,43.0,0.69,0.69,0.68,0.01,NaN,NaN,NaN,NaN
6,7,England,5.0,62.8,2.60,2.20,1.72,0.88,6.0,53.3,...,7.7,54.1,1.57,0.78,1.46,0.11,4.0,62.0,1.00,0.25
7,8,France,7.3,51.3,2.18,1.64,1.88,0.30,6.0,57.7,...,7.0,48.3,1.71,0.86,1.30,0.41,7.3,54.4,1.77,1.50
8,9,Georgia,NaN,NaN,NaN,NaN,NaN,NaN,6.0,52.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,Germany,3.0,59.3,2.00,1.67,3.35,-1.35,6.0,65.7,...,3.0,71.7,0.67,0.67,1.85,-1.18,6.3,66.8,1.11,0.95


In [4]:
#columns and axis for training
feature_columns = df.columns.difference(['squad','id']).tolist() #takes in every value except for squad and id
x = df[feature_columns]

#TODO: make a formula to determine a winner for y-axis training data

In [5]:
#dictionary containing all the groups for group stage
EURO_groups = {
    'Group A': ['Germany','Scotland','Hungary','Switzerland'],
    'Group B': ['Spain','Italy','Croatia','Albania'],
    'Group C': ['Slovenia','Denmark','Serbia','England'],
    'Group D': ['Poland','Netherlands','Austria','France'],
    'Group E': ['Belgium','Slovakia','Romania','Ukraine'],
    'Group F': ['Türkiye', 'Portugal', 'Georgia', 'Czech Republic'],
}

In [ ]:
#todo, make helper functions for simulating a match, simulating the group stage, and simulating the knockout stage


#PLACEHOLDERS

rf_model = []
scaler = []

# Unless we plan on having multiple models I don't think we need 

def simulate_match(df, team1, team2, scaler, rf_model):
    team1_stats = df.loc[df['squad'] == team1].values
    team2_stats = df.loc[df['squad'] == team2].values
    
    match_stats = np.concatenate((team1_stats, team2_stats), axis = 1)
    scaled_match_stats = scaler.transform(match_stats)
    win_probability = rf_model.predict_proba(scaled_match_stats)[0]
    if win_probability[1]>=0.6:
        return team1
    elif win_probability[1]<=0.4:
        return team2
    else:
        return ""

# Simulates results for a group
def simulate_group(t1, t2, t3, t4):
    # Initialize new dict to track points
    group = {t1: 0, t2: 0, t3: 0, t4: 0}

    # Interate to sim matches
    for team, i in group:
        for opp, j in group:
            if (team != opp):
                # ASK ABOUT WHAT TEAM_INDEX REPRESENTS
                result = simulate_match(df, team, opp, scaler, rf_model)
                if result == team:
                    group[team] += 3
                elif result == opp:
                    group[opp] += 3
                else:
                    group[team] += 1
                    group[opp] += 1

    # Sort group by points, returns decending list of tuples in the form [(team, points), ...]
    group_new = sorted(group.item(), key = lambda x: x[1], reverse=True)

    # Returns list of just teams in rank order
    teams_ranked = []
    for tuple in group_new:
        teams_ranked.append(tuple[0])

    return teams_ranked

def simulate_groupstage():
    
    groups = EURO_groups
    for group in groups:
        group_ranked = []
        group_ranked = simulate_group(group[1][0], group[1][1], group[1][2], group[1][3])
        groups.update({group: group_ranked})
    
    return groups

def knockout_stage(df,groups,scaler,rf_model):
    list39 = ['Group A','Group D','Group E','Group F']
    choice39 = random.choice(list39)
    
    list40 = list39.remove('Group A') #contains D,E,F
    if choice39 in list40:
        list40.remove(choice39)
    choice40 = random.choice(list40)
    
    list41 = ['Group A','Group B','Group C']
    if choice39 in list41:
        list41.remove(choice39)
    choice41 = random.choice(list41)
    
    list43 = ['Group A','Group B','Group C','Group D']
    if choice39 in list43:
        list43.remove(choice39)
    if choice40 in list43:
        list43.remove(choice40)
    if choice41 in list43:
        list43.remove(choice41)
    choice43 = random.choice(list43)
    
    r16 = {
        'match37': (groups['Group A'][0], groups['Group C'][1]),
        'match38': (groups['Group A'][1], groups['Group B'][1]),
        'match39': (groups['Group B'][0], groups[choice39][2]),
        'match40': (groups['Group C'][0], groups[choice40][2]),
        'match41': (groups['Group F'][0], groups[choice41][2]),
        'match42': (groups['Group D'][1], groups['Group E'][1]).
        'match43': (groups['Group E'][0], groups[choice43][1]),
        'match44': (groups['Group D'][0], groups['Group F'][1]),
    }
    
    qf = {
        'match45': (simulate_match(df,r16['match39'][0],r16['match39'][1],scaler,rf_model),simulate_match(df,r16['match37'][0],r16_['match37'][1],scaler,rf_model))
        'match46': (simulate_match(df,r16['match41'][0],r16['match41'][1],scaler,rf_model),simulate_match(df,r16['match42'][0],r16['match42'][1],scaler_rf_model))
        'match47': (simulate_match(df,r16['match43'][0],r16['match43'][1],scaler,rf_model),simulate_match(df,r16['match44'][0],r16['match44'][1],scaler,rf_model))
        'match48': (simulate_match(df,r16['match40'][0],r16['match40'][1],scaler,rf_model),simulate_match(df,r16['match38'][0],r16['match38'][1],scaler,rf_model))
    }
    
    sf = {

    'match49': (simulate_match(df, qf['match45'][0], qf['match45'][1], scaler, rf_model), simulate_match(df, qf['match46'][0], qf['match46'][1], scaler, rf_model)),
    'match50': (simulate_match(df, qf['match47'][0], qf['match47'][1], scaler, rf_model), simulate_match(df, qf['match48'][0], qf['match48'][1], scaler, rf_model)),

    }

    final = {'match51': (simulate_match(df, qf['match49'][0], qf['match49'][1], scaler, rf_model), simulate_match(df, qf['match50'][0], qf['match50'][1], scaler, rf_model))}
    
    
    
            
                 

